In [1]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os
import re

In [2]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

In [3]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel,BertTokenizer
import torch


We shall be going to work on text completion using decoder based model - GPT-2 , Falcon and encoder & decoder based model T5, flanT5 & BART

1)GPT2 token limit - 1024, 130 M parameters & 1.5 B paramters. It has comparitively less
It was needed to reduce the prompt examples from examples count 5 to example count 3 because of token limit.Even after that also
it was not generating relevant text.
GPT2  has been pre-trained in an autoregressive manner, to predict the next token in a sequence.It is good in generating coherent and relevant text
but may not be as flexible for specific task without fine tuning.

2)Falcon token limit - 2048 , 180 B parameters
Decoder based model , could not be downloaded to existing enviroment because of memory issues.Kernel got crashed.

3)T5 - 512 token limit
Encoder & Decoder based model are having better becuase they have pre-trained on various text-to-text task like translation, summarization, text completion, question
answers, text classification.T5 is more flexible to handle different type of tasks using same framework

Needed to further reduce the prompt length parameter, otherwise what happens is wherever in prompt the max length get's ended, text generation happens from
there.

4)BART - 1024 token limit
Keeping T5 limitation for token size in mind, BART was used, although it was not giving relevant solution. using Blue score 0.45

So the objective was to generate data in standard manner where all anchors that shap support are mentioned in one line.Then give explanation for only those anchors where result is very good.


### GPT2

Example 1:
Input: Mismatch Date of Birth (DOB) is good, customer date of birth is 28/10/1971 from MDM and watchlist date of birth is 00/00/1970 from Refinitv.Mismatch ID,customer ID is 686866 from MDM and watchlist ID is 14557408 from Refinitv.Match Full Legal Name, customer name is ZAW MIN  obtained from MDM and watchlist name is ZAW MIN obtained from Refinitv .Mismatch profile/occupation is good,customer is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, MANUFACTURING sector while watch person as a Director General of Prison Department of Ministry of Home Affairs. Mismatch gender. Mismatch Nationality
Output: Mismatch Date of birth and profile/occupation.Customer date of birth is 28/10/1971 and is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, while watch person  has date of birth is 00/00/1970 and is a Director General of Prison Department of Ministry of Home Affairs, appointed by the State Administration Council (SAC).

Example 2:
Input: Mismatch Date of Birth (DOB), customer date of birth is 20/08/1959 from MDM and watchlist date of birth is 00/00/1958 from Refinitv.Mismatch ID NRIC,customer NRIC is 590820016183 from
MDM and watchlist ID is 9705 from Refinitv.Mismatch Full Legal Name is good,Word count difference as customer name is JAMAL BIN SALLEH obtained
from MDM and watchlist name is JAMAL TAL ABDUL SALEH  MOHD SALEH obtained from Refinitv and Name not sounding similiar as, customer name is
JAMAL BIN SALLEH obtained from MDM and watchlist name is JAMAL TAL ABDUL SALEH obtained from Refinitv.Mismatch nationality is good.Mismatch
profile/occupation MANUFACTURING LABOURERS AT GENTING MALAYSIA BERHAD, GAMBLING/BETTING ACTIVITIES/JUNKET /CASINO/GAMING HOUSE SECTOR while watchlist is MEMBER OF
THE BOARD OF DIRECTORS OF GUANGXI GUIGANG FINANCIAL INVESTMENT AND DEVELOPMENT GROUP CO LTD (SOE) (JAN 2021 - ).

Output: Mismatch name Word count difference , Name not sounding similiar and mismatch nationality.Customer name is JAMAL BIN SALLEH and nationality
is Indian where as watch person name is JAMAL TAL ABDUL SALEH  MOHD SALEH and nationality is African

Example 3:
Input: Mismatch Date of Birth (DOB),customer date of birth is 20/08/1981 from MDM and watchlist date of birth is 04/08/1970 from Refinitv.
Mismatch ID passport is good,customer ID is K3995661 from MDM and watchlist ID is AIDPV3575A from Refinitv.
Match Full Legal Name is good, customer name is SUNIL KUMAR obtained from MDM and watchlist name is SUNIL KUMAR obtained from Refinitv.Mismatch profile/occupation is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ARAB MONETARY FUND ABU DHABI, FINANCIAL INSTITUITION SECTOR while watchperson isDIRECTOR OF COOPERATION AT THE MINISTRY OF FINANCE ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE ASIAN INFRASTRUCTURE DEVELOPMENT BANK (AIIB) ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE INTERNATIONAL BANK
FOR RECONSTRUCTION AND DEVELOPMENT (IBRD) (DEC 2020 - ). ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF INTERNATIONAL FINANCE CORPORATION (IFC) (DEC 2020 - ). ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE INTERNATIONAL DEVELOPMENT ASSOCIATION (IDA) (DEC 2020 - ). ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF MULTILATERAL INVESTMENT GUARANTEE AGENCY (MIGA) (DEC 2020 - ). ALTERNATE REPRESENTATIVE OF THE ADMINISTRATIVE COUNCIL OF INTERNATIONAL CENTRE FOR SETTLEMENT OF INVESTMENT DISPUTES (ICSID) (DEC 2020 - ). MEMBER OF THE BOARD OF EXECUTIVE DIRECTORS OF ARAB MONETARY FUND (NOV 2020 - ).

Output:Mismatch ID passport and occupation. customer has id K3995661 and works as MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ARAB MONETARY FUND ABU DHABI, FINANCIAL INSTITUITION SECTOR while watchperson is DIRECTOR OF COOPERATION AT THE MINISTRY OF FINANCE ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE ASIAN INFRASTRUCTURE DEVELOPMENT BANK (AIIB) ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE INTERNATIONAL BANK FOR RECONSTRUCTION AND DEVELOPMENT (IBRD) (DEC 2020 - ).
as while watchlist ID is AIDPV3575A and works as  DIRECTOR OF COOPERATION AT THE MINISTRY OF FINANCE ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE ASIAN INFRASTRUCTURE DEVELOPMENT BANK (AIIB)
ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE INTERNATIONAL BANK FOR RECONSTRUCTION AND DEVELOPMENT (IBRD) (DEC 2020 - ). ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF INTERNATIONAL FINANCE CORPORATION (IFC) (DEC 2020 - ). ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF THE INTERNATIONAL DEVELOPMENT ASSOCIATION (IDA) (DEC 2020 - ). ALTERNATE MEMBER OF THE BOARD OF GOVERNORS OF MULTILATERAL INVESTMENT GUARANTEE AGENCY (MIGA) (DEC 2020 - ). ALTERNATE REPRESENTATIVE OF THE ADMINISTRATIVE COUNCIL OF INTERNATIONAL CENTRE FOR SETTLEMENT OF INVESTMENT DISPUTES (ICSID) (DEC 2020 - ). MEMBER OF THE BOARD OF EXECUTIVE DIRECTORS OF ARAB MONETARY FUND (NOV 2020 - ).



Now, continue the following text:
Input: Match DOB is good,customer date of birth is 25/06/1973 from MDM and watchlist date of birth is 25/06/1973 from Refinitv.Match ID NRIC,customer NRIC is 730625145117 from MDM and watchlist NRIC is 730625145117 from Refinitv.Match occupation/profile is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR. while watch person is DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD
Output: Match DOB and occupation.Customer date of birth 25/06/1973 and works as MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD while watchlist date of birth is 25/06/1973 and works as DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD



Input:Match DOB is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/02/1988 from Refinitv.Match ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715069 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.Match occupation is good, customer works as MANAGING DIRECTORS AND CHIEF EXECUTIVES AT FORTE INSURANCE (CAMBODIA) PLC, FINANCIAL AND INSURANCE/ TAKAFUL ACTIVITIES SECTOR	MANAGING DIRECTORS AND CHIEF EXECUTIVES AT FORTE INSURANCE (CAMBODIA) PLC, FINANCIAL AND INSURANCE/ TAKAFUL ACTIVITIES SECTOR. MEMBER OF THE BOARD OF DIRECTORS OF CAMBODIAN while watch person works as MEMBER OF THE BOARD OF DIRECTORS OF CAMBODIAN REINSURANCE COMPANY (SOE) (2019 - ). GROUP CEO OF FORTE INSURANCE PLC (CAMBODIA).
Output: Match ID Nric and date of birth.Customer dob is 22/02/1988 and nric id is NRIC is 531211715069 while watchlist date of birth is 22/02/1988 and nric is 531211715069



In [4]:
prompt1 = """
Example 1:
Input: Mismatch Date of Birth (DOB) is good, customer date of birth is 28/10/1971 from MDM and watchlist date of birth is 00/00/1970 from Refinitv.Mismatch ID,customer ID is 686866 from MDM and watchlist ID is 14557408 from Refinitv.Match Full Legal Name, customer name is ZAW MIN  obtained from MDM and watchlist name is ZAW MIN obtained from Refinitv .Mismatch profile/occupation is good,customer is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, MANUFACTURING sector while watch person as a Director General of Prison Department of Ministry of Home Affairs. Mismatch gender. Mismatch Nationality
Output: Mismatch Date of birth and profile/occupation.Customer date of birth is 28/10/1971 and is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, while watch person  has date of birth is 00/00/1970 and is a Director General of Prison Department of Ministry of Home Affairs, appointed by the State Administration Council (SAC).

Example 2:
Input: Match DOB is good,customer date of birth is 25/06/1973 from MDM and watchlist date of birth is 25/06/1973 from Refinitv.Match ID NRIC,customer NRIC is 730625145117 from MDM and watchlist NRIC is 730625145117 from Refinitv.Match occupation/profile is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR. while watch person is DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD
Output: Match DOB and occupation.Customer date of birth 25/06/1973 and works as MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD while watchlist date of birth is 25/06/1973 and works as DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD


Example 3:
Input: Match DOB is good,customer date of birth is 25/06/1973 from MDM and watchlist date of birth is25/06/1973 from Refinitv.Match ID NRIC,customer NRIC is 730625145117 from MDM and watchlist NRIC is 730625145117 from Refinitv.Match occupation/profile is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR. while watch person is DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD
Output: Match DOB and occupation.Customer date of birth 25/06/1973 and works as MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD while watchlist date of birth is 25/06/1973 and works as DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD


Input:Mismatch DOB is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/06/1988 from Refinitv.Mismatch ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715068 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.
Output:
"""

In [5]:
prompt="""
Generate Output for example 3 using it's Input and by understanding how Output for example 1 and 2 is being generated using their input respectively.
All examples are completely independent of each other and output for example 3 can be generated using it's Input only.
The Input gives for any examples gives description of features - Date of Birth , ID , Name , Occupation , Gender and Nationality, whether they match or not and details of customer and watchlist.
Output contains information about only those features for which match or mismatch is good in Input followed by details of customer feature and watchlist feature for which match or mis match is good.
example 1:
Input: Mismatch Date of Birth (DOB) is good, customer date of birth is 28/10/1971 from MDM and watchlist date of birth is 00/00/1970 from Refinitv.Mismatch ID,customer ID is 686866 from MDM and watchlist ID is 14557408 from Refinitv.Match Full Legal Name, customer name is ZAW MIN  obtained from MDM and watchlist name is ZAW MIN obtained from Refinitv .Mismatch profile/occupation is good,customer is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, MANUFACTURING sector while watch person as a Director General of Prison Department of Ministry of Home Affairs. Mismatch gender. Mismatch Nationality
Output: Mismatch Date of birth and profile/occupation.Customer date of birth is 28/10/1971 and is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, while watch person  has date of birth is 00/00/1970 and is a Director General of Prison Department of Ministry of Home Affairs, appointed by the State Administration Council (SAC).

example 2:
Input: Match Date of Birth is good,customer date of birth is 25/06/1973 from MDM and watchlist date of birth is 25/06/1973 from Refinitv.Match ID NRIC,customer NRIC is 730625145117 from MDM and watchlist NRIC is 730625145117 from Refinitv.Match occupation/profile is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR, while watch person is DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD.
Output: Match Date of Birth and occupation.Customer date of birth 25/06/1973 and works as MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD while watchlist date of birth is 25/06/1973 and works as DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD


example 3:
Input:Mismatch Date of Birth is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/06/1988 from Refinitv.Mismatch ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715068 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.
Output:
"""

In [9]:
model_name = 'gpt2'  # You can also use 'gpt2-medium', 'gpt2-large', or 'gpt2-xl'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
# tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")
model = GPT2LMHeadModel.from_pretrained(model_name)
# model.eval()

D:\anaconda\Lib\site-packages\huggingface_hub\file_download.py:982: UserWarning: Not enough free disk space to download the file. The expected file size is: 548.11 MB. The target location C:\Users\MBBLABS\.cache\huggingface\hub\models--gpt2\blobs only has 47.86 MB free disk space.
  warnings.warn(


model.safetensors:  73%|#######2  | 398M/548M [00:00<?, ?B/s]

OSError: [Errno 28] No space left on device

In [7]:
# Encode the prompt
inputs = tokenizer(prompt, max_length=900,return_tensors='pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [8]:
with torch.no_grad():
     outputs = model.generate(
        inputs.input_ids,
        max_new_tokens=100,  # Maximum number of new tokens to generate
        num_beams=5,        # Beam search for better quality
        no_repeat_ngram_size=2,  # Prevent repetition
        early_stopping=True,
        temperature= 0.2,
        pad_token_id=tokenizer.eos_token_id
    )

NameError: name 'model' is not defined

In [ ]:
# outputs[0]

In [ ]:
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
generated_text

"\nGenerate Output for example 3 using it's Input and by understanding how Output for example 1 and 2 is being generated using their input respectively.\nAll examples are completely independent of each other and output for example 3 can be generated using it's Input only.\nThe Input gives for any examples gives description of features - Date of Birth, ID, Name, Occupation, Gender and Nationality, whether they match or not and details of customer and watchlist.\nOutput contains information about only those features for which match or mismatch is good in Input followed by details of customer feature and watchlist feature for which match or mis match is good.\nexample 1:\nInput: Mismatch Date of Birth (DOB) is good, customer date of birth is 28/10/1971 from MDM and watchlist date of birth is 00/00/1970 from Refinitv.Mismatch ID,customer ID is 686866 from MDM and watchlist ID is 14557408 from Refinitv.Match Full Legal Name, customer name is ZAW MIN  obtained from MDM and watchlist name is 

In [ ]:
#Falcon - cannot load model in notebook since model is very haevy and has 180 B billion parameters.

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM

# # Load the Falcon model and tokenizer
# model_name = 'falcon-model-name'  # Replace with the actual model name
# tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
# model = AutoModelForCausalLM.from_pretrained("tiiuae/falcon-7b")

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/16.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# inputs = tokenizer(prompt, return_tensors='pt') # max tokens is 2048
# # Generate the continuation
# with torch.no_grad():
#     outputs = model.generate(
#         inputs.input_ids,
#         max_new_tokens=100,  # Maximum number of new tokens to generate
#         num_beams=5,        # Beam search for better quality
#         no_repeat_ngram_size=2,  # Prevent repetition
#         early_stopping=True
#     )

# # Decode the generated text
# generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(generated_text)

Mismatch Date of Birth (DOB) is good,customer date of birth is 28/10/1971 from MDM and watchlist date is 00/00/1970 from Refinitv.Mismatch profile/occupation is also good.Customer is working as MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD while watch person is Director of ASCENSION ENGINEERING SERVICES PTE


In [ ]:
#T5 - we need to reduce prompt size more certainly otherwise since text generation shall happen from where

T5

###T5

In [ ]:
prompt="""
Generate Output for example 3 using it's Input and by understanding how Output for example 1 and 2 is being generated using Input of example 1 and example 2.
The Input gives for all three examples gives description of features - Date of Birth , ID , Name , Occupation , Gender and Nationality, whether they match or not and details of customer and watchlist.
Output contains information about only those features for which match or mismatch is good in Input followed by details of customer feature and watchlist feature for which match or mis match is good.
example 1:
Input: Mismatch Date of Birth (DOB) is good, customer date of birth is 28/10/1971 from MDM and watchlist date of birth is 00/00/1970 from Refinitv.Mismatch ID,customer ID is 686866 from MDM and watchlist ID is 14557408 from Refinitv.Match Full Legal Name, customer name is ZAW MIN  obtained from MDM and watchlist name is ZAW MIN obtained from Refinitv .Mismatch profile/occupation is good,customer is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, MANUFACTURING sector while watch person as a Director General of Prison Department of Ministry of Home Affairs. Mismatch gender. Mismatch Nationality
Output: Mismatch Date of birth and profile/occupation.Customer date of birth is 28/10/1971 and is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, while watch person  has date of birth is 00/00/1970 and is a Director General of Prison Department of Ministry of Home Affairs, appointed by the State Administration Council (SAC).

example 2:
Input: Match Date of Birth is good,customer date of birth is 25/06/1973 from MDM and watchlist date of birth is 25/06/1973 from Refinitv.Match ID NRIC,customer NRIC is 730625145117 from MDM and watchlist NRIC is 730625145117 from Refinitv.Match occupation/profile is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR, while watch person is DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD.
Output:
"""

In [ ]:
from transformers import T5Tokenizer,T5ForConditionalGeneration
model_name = 't5-large'
tokenizer_t5 = T5Tokenizer.from_pretrained(model_name)
model_t5 = T5ForConditionalGeneration.from_pretrained(model_name)

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/2.95G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
# Encode the prompt
input_ids = tokenizer_t5(prompt,max_length=900,return_tensors='pt').input_ids

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
# Generate the continuation
with torch.no_grad():
    outputs_t5 = model_t5.generate(
        input_ids,
        max_length=200,  # Maximum number of new tokens to generate
        num_beams=5,        # Beam search for better quality
        no_repeat_ngram_size=2,  # Prevent repetition
        early_stopping=True,
        pad_token_id=tokenizer_t5.eos_token_id
    )





In [ ]:
generated_text_t5 = tokenizer_t5.decode(outputs_t5[0], skip_special_tokens=True)

In [ ]:
print(generated_text_t5)

DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR, while watch person is CHIEF EXECUTIVE MANAGERS AT while while Mismatch Date of Birth is good, customer is 28/10/1971 from MDM and watchlist date of birth is 25/06/1973 from Refinitv. Example 3: Input: 1 and example 2 and.: Match,,


In [ ]:
generated_text_t5 = tokenizer_t5.decode(outputs_t5[0], skip_special_tokens=True)
generated_text_t5
print(generated_text_t5)
completion_start = generated_text_t5.find("Input:Mismatch Date of Birth is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/06/1988 from Refinitv.Mismatch ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715068 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.\nOutput:")
completion = generated_text_t5[completion_start:].strip()
# print(generated_text)
print(completion)

1 and example 2 is being generated using it's Input. The Output gives description of features - Date of Birth (DOB) is good, customer is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD.Mismatch Occupation, Gender and Nationality output contains information about customer feature and watchlist feature for which match or mismatch is bad. example 3: Inputting: Match Date and ID are good. Example
e


In [ ]:
Input:Mismatch DOB is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/06/1988 from Refinitv.Mismatch ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715068 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.
Output:

In [ ]:
#FlanT5 - Token size limit is 512
# To be used if we token sequence is more than 512, we can break data into chunks and pass the chunks one by one.

In [ ]:
### BART

### BART

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the tokenizer and model
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large')


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

In [ ]:
# Encode the prompt
input_ids = tokenizer(prompt,max_length=900,return_tensors='pt').input_ids

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [ ]:
# Generate text completion - max tokens
with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_length=100,  # Maximum number of new tokens to generate
        num_beams=5,        # Beam search for better quality
        no_repeat_ngram_size=2,  # Prevent repetition
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )


# Decode the generated text
completed_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
print(completed_text)

The Input gives for all three examples gives description of features - Date of Birth, ID, Name, Occupation, Gender and Nationality, whether they match or not and details of customer and watchlist. Example 1, 2 and 3 is being generated using Input of example 1 and example 2. Generate Output for example 3 using it's Input and by understanding how Output is generated, you will be able to understand what is happening in Output of Example 3.example 1:¯¯¯¯¯¯¯¯


In [ ]:
prompt

In [ ]:
Input:Mismatch DOB is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/06/1988 from Refinitv.Mismatch ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715068 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.
Output:

In [ ]:
# Generate the continuation - max length
with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_length=150,  # Maximum length of the generated text
        num_beams=5,     # Beam search for better quality
        no_repeat_ngram_size=2,  # Prevent repetition
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )



In [ ]:

# Decode the generated text
completed_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
completed_text

### Looping over the content

In [ ]:

import pandas as pd
df=pd.read_csv("/content/comments_gen_with_dob_name.csv")

In [ ]:

prompt_list=["""Input: Mismatch Date of Birth (DOB) is good, customer date of birth is 28/10/1971 from MDM and watchlist date of birth is 00/00/1970 from Refinitv.Mismatch ID,customer ID is 686866 from MDM and watchlist ID is 14557408 from Refinitv.Match Full Legal Name is good, customer name is ZAW MIN  obtained from MDM and watchlist name is ZAW MIN obtained from Refinitv .Mismatch profile/occupation,customer is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, MANUFACTURING sector while watch person as a Director General of Prison Department of Ministry of Home Affairs. Mismatch gender. Mismatch Nationality.\nOutput:""","""Input:Input: Match DOB is good,customer date of birth is 25/06/1973 from MDM and watchlist date of birth is 25/06/1973 from Refinitv.Match ID NRIC,customer NRIC is 730625145117 from MDM and watchlist NRIC is 730625145117 from Refinitv.Match occupation/profile is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR. while watch person is DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD.\nOutput:""","""Input:Mismatch DOB is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/06/1988 from Refinitv.Mismatch ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715068 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.\nOutput:"""]


In [ ]:
prompt_list=["""Input: Mismatch Date of Birth (DOB) is good, customer date of birth is 28/10/1971 from MDM and watchlist date of birth is 00/00/1970 from Refinitv.Mismatch ID,customer ID is 686866 from MDM and watchlist ID is 14557408 from Refinitv.Match Full Legal Name is good, customer name is ZAW MIN  obtained from MDM and watchlist name is ZAW MIN obtained from Refinitv .Mismatch profile/occupation,customer is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, MANUFACTURING sector while watch person as a Director General of Prison Department of Ministry of Home Affairs. Mismatch gender. Mismatch Nationality.\nOutput:""","""Input:Input: Match DOB is good,customer date of birth is 25/06/1973 from MDM and watchlist date of birth is 25/06/1973 from Refinitv.Match ID NRIC,customer NRIC is 730625145117 from MDM and watchlist NRIC is 730625145117 from Refinitv.Match occupation/profile is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR. while watch person is DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD.\nOutput:""","""Input:Mismatch DOB is good ,customer date of birth is 22/02/1988 from MDM and watchlist date of birth is 22/06/1988 from Refinitv.Mismatch ID NRIC is good,customer NRIC is 531211715069  from MDM and watchlist NRIC is 531211715068 from Refinitv .Mismatch Full Legal Name.Customer name is MUHAMAD SYAZWANUDDIN BIN SAHIMI obtained from MDM & watch-list name is SAHIMI MUHAMAD SYAZWANUDDIN BIN obtained from Refinitv.\nOutput:"""]
# input=tokenizer(prompt_list[0],max_length=900,return_tensors="pt")

In [ ]:
x="""Using below Input and Output of example 1 and example 2, generate output for example 3. The Input gives
description of features - date of birth , id , name , occupation , gender and nationality match or not and details of customer and watchlist
Output contains information about only those features for which match or mismatch is good followed by details of customer and then details of watchlist
example 1:
Input: Mismatch Date of Birth (DOB) is good, customer date of birth is 28/10/1971 from MDM and watchlist date of birth is 00/00/1970 from Refinitv.Mismatch ID,customer ID is 686866 from MDM and watchlist ID is 14557408 from Refinitv.Match Full Legal Name, customer name is ZAW MIN  obtained from MDM and watchlist name is ZAW MIN obtained from Refinitv .Mismatch profile/occupation is good,customer is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, MANUFACTURING sector while watch person as a Director General of Prison Department of Ministry of Home Affairs. Mismatch gender. Mismatch Nationality
Output: Mismatch Date of birth and profile/occupation.Customer date of birth is 28/10/1971 and is a MANUFACTURING LABOURERS at BEYONICS PRECISION (M) SDN BHD, while watch person  has date of birth is 00/00/1970 and is a Director General of Prison Department of Ministry of Home Affairs, appointed by the State Administration Council (SAC).

example 2:
Input: Match DOB is good,customer date of birth is 25/06/1973 from MDM and watchlist date of birth is 25/06/1973 from Refinitv.Match ID NRIC,customer NRIC is 730625145117 from MDM and watchlist NRIC is 730625145117 from Refinitv.Match occupation/profile is good, customer is MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD, CONSTRUCTION SECTOR. while watch person is DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD
Output: Match DOB and occupation.Customer date of birth 25/06/1973 and works as MANAGING DIRECTORS AND CHIEF EXECUTIVES AT ASCENSION ENGINEERING SERVICES PTE LTD while watchlist date of birth is 25/06/1973 and works as DIRECTOR OF ASCENSION ENGINEERING SERVICES PTE LTD


example 3:
"""

In [ ]:
for prompt in prompt_list[0:1]:
  # prompt=prompt+x
  input=tokenizer(prompt,max_length=900,return_tensors="pt")
  # print(input)
  outputs = model.generate(
        input.input_ids,
        max_new_tokens=100,  # Maximum length of the generated text
        num_beams=5,     # Beam search for better quality
        no_repeat_ngram_size=2,  # Prevent repetition
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id
    )
  completed_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  print(completed_text)
  # print(completed_text.split("Example 4:")[1])

Input: Mismatch Date of Birth (DOB) is good, customer date of birth is 28/10/1971 from MDM and watchlist name is 00/00/1970 from Refinitv.Match Full Legal Name,customer’s full legal name as ZAW MIN is Zaw MIN obtained from MdM, match Watchlist Name is 1/1/2018 from REFINITV and match Full Name as a Director General of Prison Department of Ministry
